In [1]:
import tensorflow as tf
import numpy as np

from IPython.display import clear_output, Image, display, HTML
from tensorflow.examples.tutorials.mnist import input_data #import input_data

In [ ]:
#################################################################################
'''
Data is downloaded as an mnist object
At every iteration, the batch of images are read from the dataset 
'''
#################################################################################
batch_size = 100
mnist = input_data.read_data_sets('../hands_on/data', one_hot=True)
def get_mnist_batch(mnist, train):
    if train:
        xs, ys = mnist.train.next_batch(batch_size)
        #k = FLAGS.dropout
    else:
        xs, ys = mnist.test.next_batch(batch_size)
        #k = 1.0
    return (2*xs)-1, ys

Extracting ../hands_on/data/train-images-idx3-ubyte.gz
Extracting ../hands_on/data/train-labels-idx1-ubyte.gz
Extracting ../hands_on/data/t10k-images-idx3-ubyte.gz
Extracting ../hands_on/data/t10k-labels-idx1-ubyte.gz


In [ ]:
#################################################################################
'''
This function creates the neural network with layer definitions
Input: Images as tensors 
Output: 10-d tensor for each image
'''
#################################################################################
def neural_network(ip):
    # Define the layers for your neural network 
    hidden_layer1 = tf.layers.dense(ip, 1296, activation=tf.nn.relu, name='hidden_layer1') 
    hidden_layer2 = tf.layers.dense(hidden_layer1, 1296, activation=tf.nn.relu, name='hidden_layer2') 
    hidden_layer3 = tf.layers.dense(hidden_layer2, 1296, activation=tf.nn.relu, name='hidden_layer3') 
    op = tf.layers.dense(hidden_layer3, 10, activation=tf.nn.relu, name='output') 
    return op


In [ ]:
#################################################################################
'''
Create Graph first by defining placeholders, ops, loss functions and optimizers

'''
#################################################################################

tf.reset_default_graph()
#################################################################################
'''Placeholders'''
#################################################################################
with tf.variable_scope('inputs'):
    inputs = tf.placeholder(tf.float32, shape=[None,784])
with tf.variable_scope('ground_truth'):
    ground_truth= tf.placeholder(tf.float32, shape=[None,10])

#################################################################################
'''MNIST Neural Network'''
#################################################################################
with tf.variable_scope('mnist_network'):
    outputs = neural_network(inputs)

    
#################################################################################
'''Loss and Optimizers'''
#################################################################################
learning_rate = 0.0001
with tf.variable_scope('cross_entropy_loss'):
    # Define the loss between the logits and the labels
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=ground_truth, logits=outputs)
    
    # Define the optimizer and minimize the loss using it
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

accuracy = []
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(outputs, 1), tf.argmax(ground_truth, 1)), tf.float32))


In [ ]:
#################################################################################
'''
Create a TF session and execute the graph

'''
#################################################################################

with tf.Session() as sess:
    # initialize all the variables - this assigns the values to the variables
    tf.global_variables_initializer().run()
    
    max_steps = 1001
    for i in range(max_steps):
        # read data from mnist dataset and get a batch
        input_, gt_ = get_mnist_batch(mnist, train=True)
    
        # The dictionary assigns the numerical values to the placeholders
        values_dict = {inputs:input_, ground_truth:gt_}
        fetch_values = [optimizer, loss, accuracy]
        #################################################################################
        '''
        Run a session to execute the graph
        sess.run() should consist of 'fetch' and 'feed' 
        fetch: are the list of variables to be returned
        feed: is the dictionary of values assigned to respective placeholders
        '''
        #################################################################################
        _, loss_, accuracy_ = sess.run(fetch_values, feed_dict=values_dict)
        
        
        if i%100==0:
            print("Accuracy at step %d: %g"%(i, accuracy_))
            #plt.imshow(input_[0].reshape([28,28]));plt.show()
            
    # Define the saver and save your model 
    saver = tf.train.Saver()
    # save the model
    saver.save(sess, 'my_mnist_model')   
    
    
        